## Plot library

This python notebook shows how to use the plot package of the Unified Planning.

[![Open In GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https:///github.com/aiplan4eu/unified-planning/blob/master/docs/notebooks/11-plot.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aiplan4eu/unified-planning/blob/master/docs/notebooks/11-plot.ipynb)


### Setup

We start by downloading (from github) the unified planning library.

In [ ]:
!pip install --pre unified-planning

We are now ready to plot some plans using the Unified-Planning library!

### Imports
We need to import all the types of plan we want to plot, the plot functions and the actions.

In [ ]:
import unified_planning
from unified_planning.plans import (
    SequentialPlan,
    TimeTriggeredPlan,
    PartialOrderPlan,
    ContingentPlanNode,
    ContingentPlan,
    STNPlanNode,
    STNPlan,
)
from unified_planning.plot import (
    plot_plan, # plot_plan plots all the types of plans, but is not customizable, while specific methods6
    plot_sequential_plan,
    plot_time_triggered_plan,
    plot_partial_order_plan,
    plot_contingent_plan,
    plot_stn_plan,
)
from unified_planning.model import InstantaneousAction, DurativeAction, TimepointKind, Fluent, Problem, Object
from unified_planning.shortcuts import BoolType, UserType, IntType, RealType

### Sequential Plan

A SequentialPlan is simply a sequence of actions.

In [ ]:
actions = list((InstantaneousAction(f"a{i}") for i in range(1, 5)))
sequential_plan = SequentialPlan([a() for a in actions])

plot_sequential_plan(sequential_plan, font_size=14)

#### Plotting fluents in SequentialPlan

On a SequentialPlan, if the UPSequentialSimulator (in the up.engines.sequential_simulator.py) supports the Problem, the value of some arbitrary expressions or the value of some quality metrics can be plotted. This shows how the value changes during the plan. 

Only numeric or boolean expressions can be plotted.

##### Define the problem

The problem defined is a robot that moves from l1 to l4, passing through l2 and l3.

While moving, the battery decreases based on the distance from the locations and the total distance accumulates.

Note: the problem has specified only the parts interesting for this simulation. It's not a complete problem to give to a planner.

In [ ]:
# Define the UserType
Location = UserType("Location")

# Define fluents
robot_at = Fluent("robot_at", BoolType(), position=Location)
battery_charge = Fluent("battery_charge", RealType(0, 100))
distance = Fluent("distance", IntType(), l_from=Location, l_to=Location)
total_distance = Fluent("total_distance", IntType())

# Define the move action
move = InstantaneousAction("move", l_from=Location, l_to=Location)
l_from = move.parameter("l_from")
l_to = move.parameter("l_to")
move.add_precondition(robot_at(l_from))
move.add_effect(robot_at(l_from), False)
move.add_effect(robot_at(l_to), True)
move.add_decrease_effect(battery_charge, distance(l_from, l_to)/2 + 5)
move.add_increase_effect(total_distance, distance(l_from, l_to))

# Define the Location Objects
objects = list((Object(f"l{i}", Location) for i in range(1, 5)))
l1, l2, l3, l4 = objects

# Create the problem, add fluents, the move action and the objects
problem = Problem("moving robot")
problem.add_fluent(robot_at, default_initial_value=False)
problem.add_fluent(battery_charge, default_initial_value=100)
problem.add_fluent(distance, default_initial_value=0)
problem.add_fluent(total_distance, default_initial_value=0)
problem.add_action(move)
problem.add_objects(objects)

# Set the initial values different from the defaults
problem.set_initial_value(robot_at(l1), True)
problem.set_initial_value(distance(l1, l2), 26)
problem.set_initial_value(distance(l2, l3), 50)
problem.set_initial_value(distance(l3, l4), 10)

# Create the plan to simulate
plan = SequentialPlan([
    move(l1, l2),
    move(l2, l3),
    move(l3, l4),
])

In [ ]:
plot_sequential_plan(plan, problem, [battery_charge, total_distance])

### TimeTriggeredPlan

A TimeTriggeredPlan is a sequence of 3 items. Every items is composed by an action, the time in which the action starts, and, if the action is a DurativeAction, the action duration.

In [ ]:
a1, a2, a3, a4 = (DurativeAction(f"a{i}") for i in range(1, 5))
time_triggered_plan = TimeTriggeredPlan([
    (0, a1(), 3),
    (1, a2(), 3),
    (2, a3(), 2),
    (2, a4(), 1),
])
# The plan can be plotted with plot_time_triggered_plan
plot_plan(time_triggered_plan)

### PartialOrderPlan

A PartialOrderPlan is a directed graph where the nodes are ActionInstances and the edges create an ordering between 2 actions.

In the unified_planning, a ParitalOrderPlan is created with the adjacency list.

Generally speaking, a SequentialPlan is a specific case of a PartialOrderPlan where every action has only one possible action that must be done before and one action that must be done later.

In [ ]:
ai1, ai2, ai3, ai4, ai5, ai6, ai7, ai8 = (InstantaneousAction(f"a{i}")() for i in range(1, 9))
partial_order_plan = PartialOrderPlan({
    ai1: [ai2],
    ai2: [ai4],
    ai3: [ai4],
    ai4: [ai5, ai6],
    ai5: [ai7],
    ai6: [ai8],
    ai7: [ai8],
})
plot_partial_order_plan(partial_order_plan, font_size=16)

### STNPlan

An STNPlan represents the temporal constraints from the events that must be performed during the plan.

An event is either the start of the plan, the end of the plan, the start of an action or the end of an action.

It is represented as a directed graph, where the nodes are the events.

There is an edge from the node N to the node M if N and M have a relative temporal constraint.
The edge is labeled with 2 numbers, the lower bound and the upper bound to the time that can elapse from N to M.

For example, N --[0, 7]--> M means that M must happen after -or at the same time, since 0 is included- N, but no more than 7 time units later.

In [ ]:
ai1, ai2, ai3, ai4 = (InstantaneousAction(f"a{i}")() for i in range(1, 5))
start_a1 = STNPlanNode(TimepointKind.START, ai1)
end_a1 = STNPlanNode(TimepointKind.END, ai1)
start_a2 = STNPlanNode(TimepointKind.START, ai2)
end_a2 = STNPlanNode(TimepointKind.END, ai2)
start_a3 = STNPlanNode(TimepointKind.START, ai3)
end_a3 = STNPlanNode(TimepointKind.END, ai3)
start_a4 = STNPlanNode(TimepointKind.START, ai4)
end_a4 = STNPlanNode(TimepointKind.END, ai4)
stn_plan = STNPlan([
    (start_a1, 1, 1, end_a1), # Link start to end actions
    (start_a2, 0, 3, end_a2),
    (start_a3, 0, 3, end_a3),
    (start_a4, 0, 3, end_a4),
    (end_a1, 0, None, start_a2), # Action 1 must finish before action 2 start (or in the same moment)
    (end_a3, None, 1, end_a4), # Action 3 can finish AT most 1 after the end of Action4
])
plot_stn_plan(stn_plan)

### ContingentPlan

A ContingentPlan is represented as a decision tree. Some actions can sense the initially unknown value of one or more fluents and decide which action to perform next based on the sensed values.

It is represented as a directed graph, with the nodes containing the action to perform and the edges labeled as the expression that must be True in order to take that branch.

In [ ]:
from unified_planning.shortcuts import *
ai1, ai2, ai3, ai4, ai5 = (InstantaneousAction(f"a{i}")() for i in range(1, 6))
node_1 = ContingentPlanNode(ai1)
node_2 = ContingentPlanNode(ai2)
node_3 = ContingentPlanNode(ai3)
node_4 = ContingentPlanNode(ai4)
node_5 = ContingentPlanNode(ai5)
x, y = Fluent("x"), Fluent("y")
node_1.add_child({x: TRUE()}, node_2)
node_1.add_child({x: FALSE()}, node_3)
node_3.add_child({y: TRUE()}, node_4)
node_3.add_child({y: FALSE()}, node_5)
contingent_plan = ContingentPlan(node_1)
plot_contingent_plan(contingent_plan, font_size = 14, edge_font_size = 12)